In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def parse_pssm(fname):
    f = open(fname)
    # the 4th line should be the start of the PSSM data
    f.readline()
    f.readline()
    f.readline()
    seq = []
    lprob = np.zeros([0,20])
    prob = np.zeros([0,20])
    extra = np.zeros([0,2])
    line = f.readline()
    while len(line.strip())>0:
        lineinfo = line.split()
        seq.append(lineinfo[1])
        lprobs_ = [float(lineinfo[i]) for i in range(2,22)]
        lprob = np.concatenate((lprob,np.matrix(lprobs_)),axis=0)
        probs_ = [float(lineinfo[i])/100 for i in range(22,42)]
        prob = np.concatenate((prob,np.matrix(probs_)),axis=0)
        extras_ = [float(lineinfo[i]) for i in range(42,44)]
        extra = np.concatenate((extra,np.matrix(extras_)),axis=0)
        line = f.readline()

    return (''.join(seq),prob,lprob,extra)

## PSSM Profile Extraction

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/HDA/Novel_BinaryML.csv')

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
pssm_matrices = []

for i in df['PDBid']:
    pssm_mat = parse_pssm(f'/content/drive/MyDrive/HDA/Novel_PSSM/{i}.txt')
    pssm_arr = np.array(pssm_mat[1])
    pssm_matrices.append(pssm_arr)

# Add the list of pssm_arr as a new column 'pssm_matrix' in the DataFrame
df['pssm_matrix'] = pssm_matrices

## Skipxgram Extraction

In [ ]:
import numpy as np

def calculate_sxgbg_features(evolutionary_profile, X):

    L, _ = evolutionary_profile.shape
    sxgbg_matrix = np.zeros((20, 20))

    for i in range(20):
        for j in range(20):
            sxgbg_value = 0.0

            for l in range(1,L - X):
                sxgbg_value += evolutionary_profile[l-1, i] * evolutionary_profile[l + X , j]

            sxgbg_matrix[i, j] = sxgbg_value

    return sxgbg_matrix

### S0G Matrix

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s0g_matrix = []

for i in df['pssm_matrix']:
    s0g_mat = calculate_sxgbg_features(i, 0)
    s0g_arr = np.array(s0g_mat)
    s0g_matrix.append(s0g_arr)

df['s0g_matrix'] = s0g_matrix

### S1G Matrix

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s1g_matrix = []

for i in df['pssm_matrix']:
    s1g_mat = calculate_sxgbg_features(i, 1)
    s1g_arr = np.array(s1g_mat)
    s1g_matrix.append(s1g_arr)

df['s1g_matrix'] = s1g_matrix

### S2G Matrix

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s2g_matrix = []

for i in df['pssm_matrix']:
    s2g_mat = calculate_sxgbg_features(i, 2)
    s2g_arr = np.array(s2g_mat)
    s2g_matrix.append(s2g_arr)

df['s2g_matrix'] = s2g_matrix

### S3G Matrix

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s3g_matrix = []

for i in df['pssm_matrix']:
    s3g_mat = calculate_sxgbg_features(i, 3)
    s3g_arr = np.array(s3g_mat)
    s3g_matrix.append(s3g_arr)

df['s3g_matrix'] = s3g_matrix

### S4G Matrix

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s4g_matrix = []

for i in df['pssm_matrix']:
    s4g_mat = calculate_sxgbg_features(i, 4)
    s4g_arr = np.array(s4g_mat)
    s4g_matrix.append(s4g_arr)

df['s4g_matrix'] = s4g_matrix

### S5G Matrix

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s5g_matrix = []

for i in df['pssm_matrix']:
    s5g_mat = calculate_sxgbg_features(i, 5)
    s5g_arr = np.array(s5g_mat)
    s5g_matrix.append(s5g_arr)

df['s5g_matrix'] = s5g_matrix

### S6G Matrix

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s6g_matrix = []

for i in df['pssm_matrix']:
    s6g_mat = calculate_sxgbg_features(i, 6)
    s6g_arr = np.array(s6g_mat)
    s6g_matrix.append(s6g_arr)

df['s6g_matrix'] = s6g_matrix

## Evaluation Metrics

In [ ]:
def calculate_accuracy(y_true, y_pred):
    # Ensure inputs are numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array((y_pred>0.5).astype(int))

    # Initialize accuracy
    accuracy = 0

    # Calculate accuracy for each instance
    for i in range(len(y_true)):
        # Calculate intersection and union
        intersection = np.sum(np.logical_and(y_true[i], y_pred[i]))
        union = np.sum(np.logical_or(y_true[i], y_pred[i]))

        # Add to total accuracy
        accuracy += intersection / union

    # Calculate average accuracy
    accuracy /= len(y_true)

    return accuracy

In [ ]:
import numpy as np

def norm_accuracy(y_true, y_pred):
    # Ensure inputs are numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array((y_pred>0.5).astype(int))

    acc = []
    # Loop over each instance
    for i in range(len(y_true)):
        # Calculate the number of correct predictions for this instance
        print(y_true[i], y_pred[i])
        correct_predictions = np.sum(y_true[i] == y_pred[i])
        print(correct_predictions/5)
        acc.append(correct_predictions/5)

    # Calculate accuracy
    accuracy = sum(acc) / len(y_true)

    return accuracy

## S0G Matrix Model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

# Define the model
model_s0g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s0g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s0g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s0g_1.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s0g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 4s 16ms/step - loss: 0.5803 - accuracy: 0.5000 - val_loss: 0.4648 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 9ms/step - loss: 0.4290 - accuracy: 0.5000 - val_loss: 0.4061 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 8ms/step - loss: 0.3748 - accuracy: 0.5625 - val_loss: 0.3608 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 5ms/step - loss: 0.3635 - accuracy: 0.5833 - val_loss: 0.3332 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 0s 6ms/step - loss: 0.3172 - accuracy: 0.6146 - val_loss: 0.3527 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 0s 5ms/step - loss: 0.2982 - accuracy: 0.6771 - val_loss: 0.4909 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 7/15
48/48 [==============================] - 0s 5ms/step - loss: 0.2760 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 14ms/step - loss: 0.4089 - accuracy: 0.5773 - val_loss: 0.2837 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.3466 - accuracy: 0.6495 - val_loss: 0.2332 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.3164 - accuracy: 0.7010 - val_loss: 0.2541 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 6ms/step - loss: 0.2682 - accuracy: 0.7526 - val_loss: 0.2108 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 5ms/step - loss: 0.2263 - accuracy: 0.8144 - val_loss: 0.1905 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 5ms/step - loss: 0.2313 - accuracy: 0.7835 - val_loss: 0.1978 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 7/15
39/49 [======================>.......] - ETA: 0s - loss: 0.1832 - accuracy: 0.871

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 8ms/step - loss: 0.3141 - accuracy: 0.7320 - val_loss: 0.2400 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.2523 - accuracy: 0.7732 - val_loss: 0.2302 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.2147 - accuracy: 0.8351 - val_loss: 0.1874 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 5ms/step - loss: 0.1923 - accuracy: 0.8247 - val_loss: 0.2087 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.2452 - accuracy: 0.7732 - val_loss: 0.2117 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 6/15
39/49 [======================>.......] - ETA: 0s - loss: 0.1512 - accuracy: 0.9231
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 5ms/step - loss: 0.1726 - accura

...

49/49 [==============================] - 2s 14ms/step - loss: 0.2289 - accuracy: 0.7732 - val_loss: 0.1463 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 9ms/step - loss: 0.2313 - accuracy: 0.8144 - val_loss: 0.1378 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 10ms/step - loss: 0.1750 - accuracy: 0.8660 - val_loss: 0.2157 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 4/15
45/49 [==========================>...] - ETA: 0s - loss: 0.2061 - accuracy: 0.8000
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 9ms/step - loss: 0.2123 - accuracy: 0.7938 - val_loss: 0.2577 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 9ms/step - loss: 0.1945 - accuracy: 0.8454 - val_loss: 0.1210 - val_accuracy: 0.9583 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 9ms/step - loss: 0.1063 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 8ms/step - loss: 0.1895 - accuracy: 0.8351 - val_loss: 0.2031 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.1674 - accuracy: 0.8763 - val_loss: 0.7214 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.1981 - accuracy: 0.8041 - val_loss: 0.2155 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 5ms/step - loss: 0.1364 - accuracy: 0.9485 - val_loss: 0.2533 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1294 - accuracy: 0.9278 - val_loss: 0.1931 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 6/15
47/49 [===========================>..] - ETA: 0s - loss: 0.1048 - accuracy: 0.8830
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.1064 - accura

1/1 [==============================] - 0s 70ms/step
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 0]
0.8
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [0 0 0 0 0]
0.8
[0 0 0 1 1] [0 0 0 1 0]
0.8
[0 1 0 0 0] [0 0 0 1 0]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 1] [0 0 0 0 0]
0.6
[0 0 0 1 0] [0 0 0 1 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [0 0 0 0 1]
0.6
[0 1 0 0 0] [0 0 0 0 0]
0.8
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
Average Accuracy:  0.6953333333333334
Accuracy:  0.9209333333333335
Average Normalized Accuracy:  0.7439444444444444
Average Precision:  0.7896101414924945
Average Recall:  0.7301318681318681
F1 score: 0.7587071016695567
Grand Mean: 0.773110037067505


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S1G Matrix Model

In [ ]:
# Define the model
model_s1g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s1g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s1g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s1g_1.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s1g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 3s 12ms/step - loss: 0.4749 - accuracy: 0.4375 - val_loss: 0.3945 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 10ms/step - loss: 0.4002 - accuracy: 0.5000 - val_loss: 0.3585 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 1s 10ms/step - loss: 0.3512 - accuracy: 0.5833 - val_loss: 0.4552 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 10ms/step - loss: 0.3368 - accuracy: 0.6771 - val_loss: 0.3201 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 0s 9ms/step - loss: 0.3097 - accuracy: 0.6979 - val_loss: 0.4900 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 1s 10ms/step - loss: 0.2952 - accuracy: 0.6771 - val_loss: 0.2924 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 7/15
48/48 [==============================] - 0s 7ms/step - loss: 0.240

1/1 [==============================] - 0s 109ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [1 0 0 0 0]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 1 1]
0.8
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [0 0 0 0 1]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 0 1]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 1 0] [0 0 0 1 0]
1.0
[0 1 0 0 0] [0 0 0 0 1]
0.6
[1 0 0 0 0] [1 0 0 0 0]
1.0
[0 0 0 0 1] [1 0 0 0 0]
0.6
[0 0 0 0 1] [0 0 0 0 1]
1.0
[0 0 0 0 1] [1 0 0 0 0]
0.6
[0 1 0 0 0] [0 0 0 0 1]
0.6
Epoch 1/15
49/49 [==============================] - 2s 19ms/step - loss: 0.2947 - accuracy: 0.8351 - val_loss: 0.0918 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 14ms/step - loss: 0.1904 - accuracy: 0.8454 - val_loss: 0.1582 - val_accuracy: 0.9167 - lr:

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
49/49 [==============================] - 2s 11ms/step - loss: 0.2207 - accuracy: 0.8660 - val_loss: 0.1868 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 9ms/step - loss: 0.2041 - accuracy: 0.8557 - val_loss: 0.1544 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 9ms/step - loss: 0.1364 - accuracy: 0.8969 - val_loss: 0.1507 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 10ms/step - loss: 0.1773 - accuracy: 0.8866 - val_loss: 0.1061 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 9ms/step - loss: 0.1013 - accuracy: 0.9072 - val_loss: 0.1193 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 7ms/step - loss: 0.1297 - accuracy: 0.8557 - val_loss: 0.1158 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 7/15
49/49 [==============================] - ETA: 0s - loss: 0.0965 - acc

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
49/49 [==============================] - 4s 14ms/step - loss: 0.1449 - accuracy: 0.8866 - val_loss: 0.1325 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 10ms/step - loss: 0.0749 - accuracy: 0.9175 - val_loss: 0.1163 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 7ms/step - loss: 0.1325 - accuracy: 0.9381 - val_loss: 0.1097 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 4/15
47/49 [===========================>..] - ETA: 0s - loss: 0.0536 - accuracy: 0.9468
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 9ms/step - loss: 0.0526 - accuracy: 0.9485 - val_loss: 0.1411 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 7ms/step - loss: 0.0697 - accuracy: 0.9381 - val_loss: 0.0886 - val_accuracy: 0.9583 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 1s 10ms/step - los

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S2G Matrix Model

In [ ]:
# Define the model
model_s2g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s2g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s2g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s2g_1.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s2g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 2s 15ms/step - loss: 0.5320 - accuracy: 0.5000 - val_loss: 0.3863 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 1s 13ms/step - loss: 0.3921 - accuracy: 0.4896 - val_loss: 0.3563 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 1s 13ms/step - loss: 0.3965 - accuracy: 0.5833 - val_loss: 0.5252 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 1s 13ms/step - loss: 0.3672 - accuracy: 0.6354 - val_loss: 0.3912 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 1s 14ms/step - loss: 0.3345 - accuracy: 0.5833 - val_loss: 0.3833 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 6/15
45/48 [===========================>..] - ETA: 0s - loss: 0.2940 - accuracy: 0.7111
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
48/48 [==============================] - 1s 15ms/step - loss

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 15ms/step - loss: 0.2425 - accuracy: 0.8144 - val_loss: 0.1765 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 10ms/step - loss: 0.1923 - accuracy: 0.8351 - val_loss: 0.1407 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 10ms/step - loss: 0.2769 - accuracy: 0.8041 - val_loss: 0.1559 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 4/15
47/49 [===========================>..] - ETA: 0s - loss: 0.1484 - accuracy: 0.8723
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 11ms/step - loss: 0.1501 - accuracy: 0.8660 - val_loss: 0.0962 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 9ms/step - loss: 0.1175 - accuracy: 0.8763 - val_loss: 0.1411 - val_accuracy: 0.8333 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 10ms/step - loss: 0.0888

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
49/49 [==============================] - 4s 16ms/step - loss: 0.3201 - accuracy: 0.7526 - val_loss: 0.1653 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 10ms/step - loss: 0.1534 - accuracy: 0.8660 - val_loss: 0.2161 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 10ms/step - loss: 0.1579 - accuracy: 0.8866 - val_loss: 0.1837 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 4/15
44/49 [=========================>....] - ETA: 0s - loss: 0.1749 - accuracy: 0.8750
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 7ms/step - loss: 0.2026 - accuracy: 0.8557 - val_loss: 0.2516 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 10ms/step - loss: 0.1405 - accuracy: 0.8454 - val_loss: 0.1498 - val_accuracy: 0.8333 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 11ms/step - loss: 0.2112 - accuracy: 0.8454 - val_loss: 0.1160 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 11ms/step - loss: 0.1925 - accuracy: 0.8144 - val_loss: 0.1232 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 13ms/step - loss: 0.1706 - accuracy: 0.7732 - val_loss: 0.1265 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - ETA: 0s - loss: 0.2107 - accuracy: 0.8351
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 29ms/step - loss: 0.2107 - accuracy: 0.8351 - val_loss: 0.1547 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 24ms/step - loss: 0.1201 - accuracy: 0.8969 - val_loss: 0.1199 - val_accuracy: 0.8750 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 1s 22ms/step - loss: 0.098

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 16ms/step - loss: 0.1432 - accuracy: 0.8763 - val_loss: 0.1007 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 11ms/step - loss: 0.1267 - accuracy: 0.8763 - val_loss: 0.1249 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 11ms/step - loss: 0.0865 - accuracy: 0.9278 - val_loss: 0.1475 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 4/15
48/49 [============================>.] - ETA: 0s - loss: 0.1362 - accuracy: 0.8750
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 11ms/step - loss: 0.1358 - accuracy: 0.8763 - val_loss: 0.1100 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 10ms/step - loss: 0.0595 - accuracy: 0.9175 - val_loss: 0.1179 - val_accuracy: 0.9167 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 1s 15ms/step - loss: 0.037

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S3G Matrix Model

In [ ]:
# Define the model
model_s3g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s3g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s3g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s3g_1.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s3g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 5s 17ms/step - loss: 0.5328 - accuracy: 0.4583 - val_loss: 0.4849 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 10ms/step - loss: 0.4299 - accuracy: 0.5208 - val_loss: 0.4969 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 9ms/step - loss: 0.3869 - accuracy: 0.5729 - val_loss: 0.3291 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 10ms/step - loss: 0.3032 - accuracy: 0.7292 - val_loss: 0.3480 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 1s 12ms/step - loss: 0.3287 - accuracy: 0.6875 - val_loss: 0.3958 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 1s 12ms/step - loss: 0.3088 - accuracy: 0.6771 - val_loss: 0.3685 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 7/15
48/48 [==============================] - 1s 12ms/step - loss: 0.24

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 13ms/step - loss: 0.3470 - accuracy: 0.7526 - val_loss: 0.2148 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 11ms/step - loss: 0.2647 - accuracy: 0.7216 - val_loss: 0.2171 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 11ms/step - loss: 0.2169 - accuracy: 0.8144 - val_loss: 0.1964 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 1s 11ms/step - loss: 0.2361 - accuracy: 0.8144 - val_loss: 0.1536 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 11ms/step - loss: 0.1731 - accuracy: 0.8866 - val_loss: 0.1210 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 1s 13ms/step - loss: 0.1308 - accuracy: 0.8866 - val_loss: 0.2198 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 7/15
49/49 [==============================] - 1s 12ms/step - loss: 0.1787 - accur

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
49/49 [==============================] - 2s 16ms/step - loss: 0.1725 - accuracy: 0.8969 - val_loss: 0.2595 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 10ms/step - loss: 0.1519 - accuracy: 0.8660 - val_loss: 0.2427 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 11ms/step - loss: 0.1530 - accuracy: 0.8247 - val_loss: 0.1487 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 4/15
47/49 [===========================>..] - ETA: 0s - loss: 0.1013 - accuracy: 0.9362
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 11ms/step - loss: 0.1001 - accuracy: 0.9278 - val_loss: 0.2489 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 10ms/step - loss: 0.1010 - accuracy: 0.8969 - val_loss: 0.1386 - val_accuracy: 0.8750 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 1s 10ms/step - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 15ms/step - loss: 0.1326 - accuracy: 0.8969 - val_loss: 0.0900 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 16ms/step - loss: 0.0903 - accuracy: 0.9381 - val_loss: 0.1154 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1080 - accuracy: 0.9072 - val_loss: 0.0481 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
39/49 [======================>.......] - ETA: 0s - loss: 0.0820 - accuracy: 0.9103
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 5ms/step - loss: 0.0770 - accuracy: 0.9278 - val_loss: 0.0602 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0515 - accuracy: 0.9381 - val_loss: 0.0999 - val_accuracy: 0.9167 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0393 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.0563 - accuracy: 0.9381 - val_loss: 0.0699 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0972 - accuracy: 0.9175 - val_loss: 0.0839 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0491 - accuracy: 0.9278 - val_loss: 1.3496 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 6ms/step - loss: 0.2282 - accuracy: 0.8969 - val_loss: 0.0925 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 5/15
45/49 [==========================>...] - ETA: 0s - loss: 0.0472 - accuracy: 0.9778
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.0615 - accuracy: 0.9485 - val_loss: 0.1074 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0357 - accura

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S4G Matrix Model

In [ ]:
# Define the model
model_s4g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s4g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s4g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s4g_1.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s4g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 1s 9ms/step - loss: 0.5400 - accuracy: 0.3854 - val_loss: 0.4891 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 5ms/step - loss: 0.4129 - accuracy: 0.4792 - val_loss: 0.4455 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 5ms/step - loss: 0.3671 - accuracy: 0.6146 - val_loss: 0.4956 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 5ms/step - loss: 0.3304 - accuracy: 0.6875 - val_loss: 0.3689 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 0s 5ms/step - loss: 0.2827 - accuracy: 0.7292 - val_loss: 0.3722 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 0s 7ms/step - loss: 0.2751 - accuracy: 0.7917 - val_loss: 0.3613 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 7/15
48/48 [==============================] - 0s 9ms/step - loss: 0.2629 - a

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
49/49 [==============================] - 2s 8ms/step - loss: 0.3783 - accuracy: 0.6289 - val_loss: 0.4262 - val_accuracy: 0.2917 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.3392 - accuracy: 0.6392 - val_loss: 0.2886 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.2849 - accuracy: 0.7938 - val_loss: 0.2590 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 5ms/step - loss: 0.2477 - accuracy: 0.7938 - val_loss: 0.2169 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
41/49 [========================>.....] - ETA: 0s - loss: 0.2072 - accuracy: 0.8171
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 5ms/step - loss: 0.2319 - accuracy: 0.7938 - val_loss: 0.5504 - val_accuracy: 0.3750 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.18

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 8ms/step - loss: 0.3268 - accuracy: 0.7320 - val_loss: 0.2183 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.2376 - accuracy: 0.7835 - val_loss: 0.2215 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.2236 - accuracy: 0.8041 - val_loss: 0.3199 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 5ms/step - loss: 0.1870 - accuracy: 0.8557 - val_loss: 0.2375 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 5ms/step - loss: 0.1908 - accuracy: 0.8041 - val_loss: 0.2117 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1530 - accuracy: 0.8557 - val_loss: 0.2373 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 7/15
49/49 [==============================] - 0s 5ms/step - loss: 0.1193 - accuracy: 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 8ms/step - loss: 0.1968 - accuracy: 0.8660 - val_loss: 0.1128 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.1872 - accuracy: 0.8144 - val_loss: 0.1462 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.1496 - accuracy: 0.8763 - val_loss: 0.1704 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 5ms/step - loss: 0.1863 - accuracy: 0.8144 - val_loss: 0.1304 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 5ms/step - loss: 0.1397 - accuracy: 0.8660 - val_loss: 0.1360 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0935 - accuracy: 0.8969 - val_loss: 0.1613 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 7/15
46/49 [===========================>..] - ETA: 0s - loss: 0.1592 - accuracy: 0.8370

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
49/49 [==============================] - 3s 28ms/step - loss: 0.1605 - accuracy: 0.8454 - val_loss: 0.1541 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 8ms/step - loss: 0.1280 - accuracy: 0.8763 - val_loss: 0.1292 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 8ms/step - loss: 0.0846 - accuracy: 0.9175 - val_loss: 0.1985 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 4/15
43/49 [=========================>....] - ETA: 0s - loss: 0.1024 - accuracy: 0.8721
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 7ms/step - loss: 0.0988 - accuracy: 0.8763 - val_loss: 0.1387 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 10ms/step - loss: 0.0653 - accuracy: 0.9588 - val_loss: 0.1764 - val_accuracy: 0.7917 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 1s 11ms/step - los

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S5G Matrix Model

In [ ]:
# Define the model
model_s5g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s5g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s5g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s5g_1.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s5g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 2s 18ms/step - loss: 0.4581 - accuracy: 0.4583 - val_loss: 0.6471 - val_accuracy: 0.1600 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 7ms/step - loss: 0.4120 - accuracy: 0.4896 - val_loss: 0.4150 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 7ms/step - loss: 0.3571 - accuracy: 0.6042 - val_loss: 0.3178 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 8ms/step - loss: 0.3372 - accuracy: 0.6562 - val_loss: 0.3280 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 0s 8ms/step - loss: 0.3009 - accuracy: 0.6979 - val_loss: 0.2746 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 0s 10ms/step - loss: 0.3104 - accuracy: 0.6771 - val_loss: 0.2934 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 7/15
48/48 [==============================] - 0s 10ms/step - loss: 0.2255 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
49/49 [==============================] - 3s 22ms/step - loss: 0.2899 - accuracy: 0.7010 - val_loss: 0.1848 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 10ms/step - loss: 0.2548 - accuracy: 0.7732 - val_loss: 0.4272 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 14ms/step - loss: 0.2514 - accuracy: 0.7629 - val_loss: 0.2863 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 8ms/step - loss: 0.2281 - accuracy: 0.7835 - val_loss: 0.2845 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 9ms/step - loss: 0.2152 - accuracy: 0.8247 - val_loss: 0.2087 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.2119 - accuracy: 0.8041 - val_loss: 0.2347 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 7/15
47/49 [===========================>..] - ETA: 0s - loss: 0.1652 - ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 14ms/step - loss: 0.2316 - accuracy: 0.7732 - val_loss: 0.1572 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 8ms/step - loss: 0.2357 - accuracy: 0.7938 - val_loss: 0.2509 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 8ms/step - loss: 0.1718 - accuracy: 0.8660 - val_loss: 0.2345 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - ETA: 0s - loss: 0.1566 - accuracy: 0.8763
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 10ms/step - loss: 0.1566 - accuracy: 0.8763 - val_loss: 0.2216 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 11ms/step - loss: 0.1190 - accuracy: 0.9278 - val_loss: 0.1625 - val_accuracy: 0.7917 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 1s 14ms/step - loss: 0.1361 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 3s 16ms/step - loss: 0.1833 - accuracy: 0.8041 - val_loss: 0.1721 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 10ms/step - loss: 0.1626 - accuracy: 0.8557 - val_loss: 0.1698 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 9ms/step - loss: 0.1401 - accuracy: 0.8763 - val_loss: 0.1362 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 1s 11ms/step - loss: 0.1011 - accuracy: 0.8763 - val_loss: 0.0886 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 9ms/step - loss: 0.0998 - accuracy: 0.8969 - val_loss: 0.0749 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 1s 11ms/step - loss: 0.0934 - accuracy: 0.8763 - val_loss: 0.1645 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 7/15
49/49 [==============================] - 1s 11ms/step - loss: 0.1085 - accurac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 8ms/step - loss: 0.2180 - accuracy: 0.8351 - val_loss: 0.1923 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0956 - accuracy: 0.9072 - val_loss: 0.1096 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0962 - accuracy: 0.9072 - val_loss: 0.2596 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0935 - accuracy: 0.8969 - val_loss: 0.1066 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 5/15
37/49 [=====================>........] - ETA: 0s - loss: 0.0947 - accuracy: 0.9459
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.0815 - accuracy: 0.9485 - val_loss: 0.0997 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0353 - accura

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S6G Matrix Model

In [ ]:
# Define the model
model_s6g_1 = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s6g_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s6g_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s6g_1.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s6g_1.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 1s 9ms/step - loss: 0.5433 - accuracy: 0.5104 - val_loss: 0.4343 - val_accuracy: 0.6000 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 6ms/step - loss: 0.3973 - accuracy: 0.5833 - val_loss: 0.3924 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 5ms/step - loss: 0.3786 - accuracy: 0.5833 - val_loss: 0.3723 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 5ms/step - loss: 0.3659 - accuracy: 0.6042 - val_loss: 0.3044 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 0s 7ms/step - loss: 0.2939 - accuracy: 0.6667 - val_loss: 0.3222 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 1s 11ms/step - loss: 0.2840 - accuracy: 0.6771 - val_loss: 0.2987 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 7/15
47/48 [============================>.] - ETA: 0s - loss: 0.2534 - accu

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
49/49 [==============================] - 3s 13ms/step - loss: 0.3970 - accuracy: 0.6289 - val_loss: 0.3117 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 11ms/step - loss: 0.2898 - accuracy: 0.6907 - val_loss: 0.3135 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 11ms/step - loss: 0.2746 - accuracy: 0.7423 - val_loss: 0.3157 - val_accuracy: 0.5833 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 9ms/step - loss: 0.2739 - accuracy: 0.7320 - val_loss: 0.2040 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 10ms/step - loss: 0.2321 - accuracy: 0.7629 - val_loss: 0.2388 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 1s 11ms/step - loss: 0.3042 - accuracy: 0.6598 - val_loss: 0.2163 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 7/15
49/49 [==============================] - ETA: 0s - loss: 0.2044 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
49/49 [==============================] - 2s 13ms/step - loss: 0.2583 - accuracy: 0.7629 - val_loss: 0.1894 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 1s 12ms/step - loss: 0.2489 - accuracy: 0.7938 - val_loss: 0.2783 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 11ms/step - loss: 0.1892 - accuracy: 0.8144 - val_loss: 0.2177 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 4/15
44/49 [=========================>....] - ETA: 0s - loss: 0.2181 - accuracy: 0.8182
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 12ms/step - loss: 0.2108 - accuracy: 0.8351 - val_loss: 0.2614 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 18ms/step - loss: 0.1795 - accuracy: 0.8660 - val_loss: 0.2071 - val_accuracy: 0.8750 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 1s 21ms/step - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 11ms/step - loss: 0.2284 - accuracy: 0.8351 - val_loss: 0.1854 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 9ms/step - loss: 0.1652 - accuracy: 0.8454 - val_loss: 0.1299 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 9ms/step - loss: 0.1465 - accuracy: 0.8763 - val_loss: 0.1317 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 10ms/step - loss: 0.1268 - accuracy: 0.8866 - val_loss: 0.1751 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 10ms/step - loss: 0.1267 - accuracy: 0.8866 - val_loss: 0.1086 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 6/15
46/49 [===========================>..] - ETA: 0s - loss: 0.1116 - accuracy: 0.8696
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 11ms/step - loss: 0.1135 - ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 13ms/step - loss: 0.1508 - accuracy: 0.8660 - val_loss: 0.1261 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 7ms/step - loss: 0.1454 - accuracy: 0.8969 - val_loss: 0.1718 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 1s 14ms/step - loss: 0.1885 - accuracy: 0.8247 - val_loss: 0.2206 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 4/15
45/49 [==========================>...] - ETA: 0s - loss: 0.1110 - accuracy: 0.8667
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 1s 12ms/step - loss: 0.1050 - accuracy: 0.8763 - val_loss: 0.1938 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 14ms/step - loss: 0.0764 - accuracy: 0.9072 - val_loss: 0.1567 - val_accuracy: 0.8333 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 1s 13ms/step - loss: 0.0547

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Standardizing SkipGram

In [ ]:
# standardize the sxgbg_matrix
import numpy as np

def calculate_sxgbg_stan_features(evolutionary_profile, X):
    L, _ = evolutionary_profile.shape
    sxgbg_matrix = np.zeros((20, 20))

    for i in range(20):
        for j in range(20):
            sxgbg_value = 0.0

            for l in range(1, L - X):
                sxgbg_value += evolutionary_profile[l - 1, i] * evolutionary_profile[l + X, j]

            sxgbg_matrix[i, j] = sxgbg_value

    # Standardize the sxgbg_matrix (z-score normalization)
    mean = np.mean(sxgbg_matrix)
    std = np.std(sxgbg_matrix)

    if std != 0:
        sxgbg_matrix = (sxgbg_matrix - mean) / std
    else:
        sxgbg_matrix = np.zeros_like(sxgbg_matrix)  # Handle the case of zero standard deviation

    return sxgbg_matrix

## S0G Stan

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s0g_stan_matrix = []

for i in df['pssm_matrix']:
    s0g_mat = calculate_sxgbg_stan_features(i, 0)
    s0g_arr = np.array(s0g_mat)
    s0g_stan_matrix.append(s0g_arr)

df['s0g_stan_matrix'] = s0g_stan_matrix

## S1G Stan

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s1g_stan_matrix = []

for i in df['pssm_matrix']:
    s1g_mat = calculate_sxgbg_stan_features(i, 1)
    s1g_arr = np.array(s1g_mat)
    s1g_stan_matrix.append(s1g_arr)

df['s1g_stan_matrix'] = s1g_stan_matrix

## S2G Stan

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s2g_stan_matrix = []

for i in df['pssm_matrix']:
    s2g_mat = calculate_sxgbg_stan_features(i, 2)
    s2g_arr = np.array(s2g_mat)
    s2g_stan_matrix.append(s2g_arr)

df['s2g_stan_matrix'] = s2g_stan_matrix

## S3G Stan

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s3g_stan_matrix = []

for i in df['pssm_matrix']:
    s3g_mat = calculate_sxgbg_stan_features(i, 3)
    s3g_arr = np.array(s3g_mat)
    s3g_stan_matrix.append(s3g_arr)

df['s3g_stan_matrix'] = s3g_stan_matrix

## S4G Stan

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s4g_stan_matrix = []

for i in df['pssm_matrix']:
    s4g_mat = calculate_sxgbg_stan_features(i, 4)
    s4g_arr = np.array(s4g_mat)
    s4g_stan_matrix.append(s4g_arr)

df['s4g_stan_matrix'] = s4g_stan_matrix

## S5G Stan

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s5g_stan_matrix = []

for i in df['pssm_matrix']:
    s5g_mat = calculate_sxgbg_stan_features(i, 5)
    s5g_arr = np.array(s5g_mat)
    s5g_stan_matrix.append(s5g_arr)

df['s5g_stan_matrix'] = s5g_stan_matrix

## S6G Stan

In [ ]:
import numpy as np

# Create an empty list to store pssm_arr for each PDBid
s6g_stan_matrix = []

for i in df['pssm_matrix']:
    s6g_mat = calculate_sxgbg_stan_features(i, 6)
    s6g_arr = np.array(s6g_mat)
    s6g_stan_matrix.append(s6g_arr)

df['s6g_stan_matrix'] = s6g_stan_matrix

## S0G Stan Matrix Model

In [ ]:
# Define the model
model_s0g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s0g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s0g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s0g_stan.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s0g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 3s 19ms/step - loss: 0.4334 - accuracy: 0.4688 - val_loss: 0.4058 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 5ms/step - loss: 0.2904 - accuracy: 0.7083 - val_loss: 0.2595 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 5ms/step - loss: 0.2004 - accuracy: 0.8542 - val_loss: 0.2989 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 5ms/step - loss: 0.1540 - accuracy: 0.8646 - val_loss: 0.2831 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 0s 5ms/step - loss: 0.1213 - accuracy: 0.9167 - val_loss: 0.2937 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 0s 5ms/step - loss: 0.0647 - accuracy: 0.9792 - val_loss: 0.2971 - val_accuracy: 0.8400 - lr: 0.0010
Epoch 7/15
48/48 [==============================] - 0s 5ms/step - loss: 0.0774 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.1498 - accuracy: 0.8763 - val_loss: 0.0407 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.1460 - accuracy: 0.8454 - val_loss: 0.1371 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.1069 - accuracy: 0.9381 - val_loss: 0.0679 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - ETA: 0s - loss: 0.0546 - accuracy: 0.9072
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 5ms/step - loss: 0.0546 - accuracy: 0.9072 - val_loss: 0.0538 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0179 - accuracy: 0.9278 - val_loss: 0.0371 - val_accuracy: 0.9167 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0110 - ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 8ms/step - loss: 0.1391 - accuracy: 0.9072 - val_loss: 0.0975 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.1035 - accuracy: 0.9175 - val_loss: 0.0492 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 8ms/step - loss: 0.0712 - accuracy: 0.9381 - val_loss: 0.0433 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 9ms/step - loss: 0.0470 - accuracy: 0.9588 - val_loss: 0.0891 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 10ms/step - loss: 0.0207 - accuracy: 0.9588 - val_loss: 0.0625 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 6/15
43/49 [=========================>....] - ETA: 0s - loss: 0.0134 - accuracy: 0.9767
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 10ms/step - loss: 0.0130 - accu

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.1017 - accuracy: 0.8969 - val_loss: 0.0367 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0388 - accuracy: 0.9485 - val_loss: 0.3662 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0993 - accuracy: 0.8763 - val_loss: 0.0208 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0380 - accuracy: 0.9381 - val_loss: 0.0288 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0315 - accuracy: 0.9485 - val_loss: 0.0383 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 6/15
37/49 [=====================>........] - ETA: 0s - loss: 0.0099 - accuracy: 0.9459
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 5ms/step - loss: 0.0116 - accura

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 8ms/step - loss: 0.0509 - accuracy: 0.9485 - val_loss: 0.1207 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0517 - accuracy: 0.9485 - val_loss: 0.0771 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0274 - accuracy: 0.9485 - val_loss: 0.1676 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 1s 17ms/step - loss: 0.0062 - accuracy: 0.9588 - val_loss: 0.0290 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0029 - accuracy: 0.9381 - val_loss: 0.0511 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0017 - accuracy: 0.9485 - val_loss: 0.0286 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 7/15
38/49 [======================>.......] - ETA: 0s - loss: 6.1354e-04 - accuracy: 0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S1G Stan Matrix Model

In [ ]:
# Define the model
model_s1g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s1g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s1g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s1g_stan.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s1g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 2s 15ms/step - loss: 0.4339 - accuracy: 0.5000 - val_loss: 0.4031 - val_accuracy: 0.3600 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 10ms/step - loss: 0.3060 - accuracy: 0.6875 - val_loss: 0.2844 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 5ms/step - loss: 0.2169 - accuracy: 0.7812 - val_loss: 0.2529 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 6ms/step - loss: 0.1641 - accuracy: 0.8333 - val_loss: 0.2634 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 5/15
38/48 [======================>.......] - ETA: 0s - loss: 0.0925 - accuracy: 0.9474
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
48/48 [==============================] - 0s 5ms/step - loss: 0.1189 - accuracy: 0.9167 - val_loss: 0.2826 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 0s 5ms/step - loss: 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.1714 - accuracy: 0.8763 - val_loss: 0.0433 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0921 - accuracy: 0.8763 - val_loss: 0.0910 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0741 - accuracy: 0.8866 - val_loss: 0.0678 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 4/15
40/49 [=======================>......] - ETA: 0s - loss: 0.0482 - accuracy: 0.9125
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 5ms/step - loss: 0.0456 - accuracy: 0.9072 - val_loss: 0.0512 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0177 - accuracy: 0.9485 - val_loss: 0.0495 - val_accuracy: 0.9167 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0087 - ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 8ms/step - loss: 0.1457 - accuracy: 0.8351 - val_loss: 0.0879 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0850 - accuracy: 0.9278 - val_loss: 0.0699 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.1046 - accuracy: 0.8763 - val_loss: 0.0815 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
48/49 [============================>.] - ETA: 0s - loss: 0.0460 - accuracy: 0.9375
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 7ms/step - loss: 0.0456 - accuracy: 0.9381 - val_loss: 0.0659 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 9ms/step - loss: 0.0204 - accuracy: 0.9691 - val_loss: 0.0969 - val_accuracy: 0.9167 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 9ms/step - loss: 0.0125 - ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 15ms/step - loss: 0.1277 - accuracy: 0.8660 - val_loss: 0.0377 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0846 - accuracy: 0.9072 - val_loss: 0.0359 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0678 - accuracy: 0.8969 - val_loss: 0.0625 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 4/15
47/49 [===========================>..] - ETA: 0s - loss: 0.0631 - accuracy: 0.9255
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 5ms/step - loss: 0.0651 - accuracy: 0.9278 - val_loss: 0.0673 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0461 - accuracy: 0.8969 - val_loss: 0.0116 - val_accuracy: 0.9583 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0116 - a

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.1122 - accuracy: 0.8763 - val_loss: 0.0614 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0648 - accuracy: 0.8969 - val_loss: 0.1607 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0431 - accuracy: 0.9381 - val_loss: 0.0631 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
47/49 [===========================>..] - ETA: 0s - loss: 0.0205 - accuracy: 0.9362
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.0201 - accuracy: 0.9381 - val_loss: 0.1964 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0131 - accuracy: 0.9278 - val_loss: 0.0812 - val_accuracy: 0.9167 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0035 - ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S2G Stan Matrix Model

In [ ]:
# Define the model
model_s2g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s2g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s2g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s2g_stan.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s2g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 1s 9ms/step - loss: 0.4459 - accuracy: 0.4792 - val_loss: 0.2833 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 5ms/step - loss: 0.2363 - accuracy: 0.7604 - val_loss: 0.2861 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 6ms/step - loss: 0.1447 - accuracy: 0.8438 - val_loss: 0.2908 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 6ms/step - loss: 0.1157 - accuracy: 0.9271 - val_loss: 0.2983 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 0s 5ms/step - loss: 0.1094 - accuracy: 0.8438 - val_loss: 0.2709 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 0s 6ms/step - loss: 0.0758 - accuracy: 0.8958 - val_loss: 0.2862 - val_accuracy: 0.8400 - lr: 0.0010
Epoch 7/15
48/48 [==============================] - 0s 6ms/step - loss: 0.0536 - a

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 15ms/step - loss: 0.1602 - accuracy: 0.8557 - val_loss: 0.0474 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 10ms/step - loss: 0.0990 - accuracy: 0.9381 - val_loss: 0.0581 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 10ms/step - loss: 0.0804 - accuracy: 0.9175 - val_loss: 0.1125 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 4/15
43/49 [=========================>....] - ETA: 0s - loss: 0.0767 - accuracy: 0.9070
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 10ms/step - loss: 0.0792 - accuracy: 0.9072 - val_loss: 0.1235 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 1s 11ms/step - loss: 0.0373 - accuracy: 0.9485 - val_loss: 0.0534 - val_accuracy: 0.9167 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 1s 11ms/step - loss: 0.019

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 15ms/step - loss: 0.1457 - accuracy: 0.8969 - val_loss: 0.0321 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 10ms/step - loss: 0.0802 - accuracy: 0.9485 - val_loss: 0.0627 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 9ms/step - loss: 0.0611 - accuracy: 0.9381 - val_loss: 0.1003 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 4/15
39/49 [======================>.......] - ETA: 0s - loss: 0.0519 - accuracy: 0.9231
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.0457 - accuracy: 0.9381 - val_loss: 0.0740 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0215 - accuracy: 0.9485 - val_loss: 0.0453 - val_accuracy: 0.9583 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0079 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 8ms/step - loss: 0.0598 - accuracy: 0.9278 - val_loss: 0.0625 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0526 - accuracy: 0.9381 - val_loss: 0.0097 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0612 - accuracy: 0.9072 - val_loss: 0.0379 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
46/49 [===========================>..] - ETA: 0s - loss: 0.0198 - accuracy: 0.9457
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.0189 - accuracy: 0.9485 - val_loss: 0.0134 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0031 - accuracy: 0.9278 - val_loss: 0.0040 - val_accuracy: 0.9167 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0016 - ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 15ms/step - loss: 0.0723 - accuracy: 0.9175 - val_loss: 0.0206 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 10ms/step - loss: 0.0339 - accuracy: 0.9381 - val_loss: 0.1117 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 10ms/step - loss: 0.0329 - accuracy: 0.9691 - val_loss: 0.0122 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 4/15
44/49 [=========================>....] - ETA: 0s - loss: 0.0339 - accuracy: 0.9545
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 7ms/step - loss: 0.0377 - accuracy: 0.9485 - val_loss: 0.0864 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0151 - accuracy: 0.9485 - val_loss: 0.0233 - val_accuracy: 0.9583 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0030 -

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S3G Stan Matrix Model

In [ ]:
# Define the model
model_s3g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s3g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s3g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s3g_stan.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s3g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 1s 9ms/step - loss: 0.4263 - accuracy: 0.5417 - val_loss: 0.4002 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 5ms/step - loss: 0.2995 - accuracy: 0.6979 - val_loss: 0.3286 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 5ms/step - loss: 0.2193 - accuracy: 0.8125 - val_loss: 0.2586 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 5ms/step - loss: 0.1671 - accuracy: 0.8750 - val_loss: 0.2600 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 0s 5ms/step - loss: 0.1169 - accuracy: 0.8854 - val_loss: 0.2664 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 6/15
42/48 [=========================>....] - ETA: 0s - loss: 0.0847 - accuracy: 0.9524
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
48/48 [==============================] - 0s 5ms/step - loss: 0.09

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.1533 - accuracy: 0.8866 - val_loss: 0.1865 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.1159 - accuracy: 0.8763 - val_loss: 0.0676 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0515 - accuracy: 0.9278 - val_loss: 0.0809 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0438 - accuracy: 0.9381 - val_loss: 0.0420 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0187 - accuracy: 0.9485 - val_loss: 0.0658 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0092 - accuracy: 0.9485 - val_loss: 0.0346 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 7/15
46/49 [===========================>..] - ETA: 0s - loss: 0.0052 - accuracy: 0.9674

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 14ms/step - loss: 0.0803 - accuracy: 0.9175 - val_loss: 0.0230 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 10ms/step - loss: 0.0537 - accuracy: 0.9175 - val_loss: 0.0213 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0382 - accuracy: 0.9485 - val_loss: 0.0402 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - ETA: 0s - loss: 0.0221 - accuracy: 0.9485
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 5ms/step - loss: 0.0221 - accuracy: 0.9485 - val_loss: 0.0789 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0120 - accuracy: 0.9691 - val_loss: 0.0300 - val_accuracy: 0.8750 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0038 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.0729 - accuracy: 0.9485 - val_loss: 0.0717 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0365 - accuracy: 0.9381 - val_loss: 0.0143 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0226 - accuracy: 0.9485 - val_loss: 0.0049 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0304 - accuracy: 0.9381 - val_loss: 0.0654 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
48/49 [============================>.] - ETA: 0s - loss: 0.0435 - accuracy: 0.9375
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.0430 - accuracy: 0.9381 - val_loss: 0.0644 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0260 - accura

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 8ms/step - loss: 0.0655 - accuracy: 0.9278 - val_loss: 0.0137 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0172 - accuracy: 0.9485 - val_loss: 0.0419 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0197 - accuracy: 0.9588 - val_loss: 0.0170 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
41/49 [========================>.....] - ETA: 0s - loss: 0.0276 - accuracy: 0.9512
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 5ms/step - loss: 0.0411 - accuracy: 0.9381 - val_loss: 0.0188 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0153 - accuracy: 0.9485 - val_loss: 0.0181 - val_accuracy: 0.9583 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0046 - ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S4G Stan Matrix Model

In [ ]:
# Define the model
model_s4g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s4g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s4g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s4g_stan.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s4g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 2s 14ms/step - loss: 0.4206 - accuracy: 0.5000 - val_loss: 0.4106 - val_accuracy: 0.4400 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 9ms/step - loss: 0.2894 - accuracy: 0.6979 - val_loss: 0.2667 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 8ms/step - loss: 0.2000 - accuracy: 0.8125 - val_loss: 0.2849 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 10ms/step - loss: 0.1447 - accuracy: 0.8646 - val_loss: 0.3437 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 5/15
47/48 [============================>.] - ETA: 0s - loss: 0.1085 - accuracy: 0.8617
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
48/48 [==============================] - 0s 9ms/step - loss: 0.1088 - accuracy: 0.8646 - val_loss: 0.2920 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 0s 9ms/step - loss: 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.1719 - accuracy: 0.8247 - val_loss: 0.1042 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1286 - accuracy: 0.8660 - val_loss: 0.2279 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.1099 - accuracy: 0.8763 - val_loss: 0.0957 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 4/15
40/49 [=======================>......] - ETA: 0s - loss: 0.0692 - accuracy: 0.8750
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 5ms/step - loss: 0.0666 - accuracy: 0.8866 - val_loss: 0.0595 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0382 - accuracy: 0.9072 - val_loss: 0.0661 - val_accuracy: 0.9583 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0263 - ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 8ms/step - loss: 0.0939 - accuracy: 0.9175 - val_loss: 0.0650 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0605 - accuracy: 0.9381 - val_loss: 0.0650 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0511 - accuracy: 0.9381 - val_loss: 0.0147 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0464 - accuracy: 0.9485 - val_loss: 0.0208 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
46/49 [===========================>..] - ETA: 0s - loss: 0.0258 - accuracy: 0.9457
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 7ms/step - loss: 0.0247 - accuracy: 0.9485 - val_loss: 0.1859 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 9ms/step - loss: 0.0253 - accura

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/15
49/49 [==============================] - 2s 11ms/step - loss: 0.0863 - accuracy: 0.8866 - val_loss: 0.0110 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0453 - accuracy: 0.9175 - val_loss: 0.0432 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0273 - accuracy: 0.9381 - val_loss: 0.0220 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
38/49 [======================>.......] - ETA: 0s - loss: 0.0074 - accuracy: 0.9474
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 5ms/step - loss: 0.0080 - accuracy: 0.9485 - val_loss: 0.0212 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0060 - accuracy: 0.9381 - val_loss: 0.0383 - val_accuracy: 0.9583 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 5ms/step - loss:

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.0821 - accuracy: 0.9072 - val_loss: 0.0284 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0206 - accuracy: 0.9485 - val_loss: 0.0480 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0567 - accuracy: 0.9278 - val_loss: 0.0679 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - ETA: 0s - loss: 0.0333 - accuracy: 0.9588
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 5ms/step - loss: 0.0333 - accuracy: 0.9588 - val_loss: 0.1998 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0155 - accuracy: 0.9381 - val_loss: 0.0783 - val_accuracy: 0.8750 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0056 - ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S5G Stan Matrix Model

In [ ]:
# Define the model
model_s5g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s5g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s5g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s5g_stan.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s5g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 1s 9ms/step - loss: 0.4336 - accuracy: 0.4167 - val_loss: 0.3047 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 5ms/step - loss: 0.2644 - accuracy: 0.7396 - val_loss: 0.2936 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 5ms/step - loss: 0.1786 - accuracy: 0.8750 - val_loss: 0.2671 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 6ms/step - loss: 0.1297 - accuracy: 0.8958 - val_loss: 0.2852 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 0s 5ms/step - loss: 0.1471 - accuracy: 0.8438 - val_loss: 0.2597 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 6/15
39/48 [=======================>......] - ETA: 0s - loss: 0.0793 - accuracy: 0.9487
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
48/48 [==============================] - 0s 5ms/step - loss: 0.08

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 14ms/step - loss: 0.2491 - accuracy: 0.7320 - val_loss: 0.1254 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 10ms/step - loss: 0.1762 - accuracy: 0.8557 - val_loss: 0.1171 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 9ms/step - loss: 0.1252 - accuracy: 0.8660 - val_loss: 0.1572 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 10ms/step - loss: 0.0930 - accuracy: 0.8866 - val_loss: 0.1039 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 9ms/step - loss: 0.0698 - accuracy: 0.9278 - val_loss: 0.1030 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 9ms/step - loss: 0.0465 - accuracy: 0.9278 - val_loss: 0.1091 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 7/15
38/49 [======================>.......] - ETA: 0s - loss: 0.0329 - accuracy: 0.9

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 8ms/step - loss: 0.1479 - accuracy: 0.8866 - val_loss: 0.1802 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1071 - accuracy: 0.9175 - val_loss: 0.0570 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0604 - accuracy: 0.9381 - val_loss: 0.0539 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0330 - accuracy: 0.9485 - val_loss: 0.2779 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 6ms/step - loss: 0.0400 - accuracy: 0.9897 - val_loss: 0.1439 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 6/15
37/49 [=====================>........] - ETA: 0s - loss: 0.0238 - accuracy: 0.9324
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 5ms/step - loss: 0.0233 - accura

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.0762 - accuracy: 0.9381 - val_loss: 0.1585 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0591 - accuracy: 0.9381 - val_loss: 0.0288 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0305 - accuracy: 0.9278 - val_loss: 0.0389 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0402 - accuracy: 0.9381 - val_loss: 0.3244 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0493 - accuracy: 0.9485 - val_loss: 0.0335 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 6/15
48/49 [============================>.] - ETA: 0s - loss: 0.0359 - accuracy: 0.9375
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 5ms/step - loss: 0.0356 - accura

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 15ms/step - loss: 0.0580 - accuracy: 0.9485 - val_loss: 0.0271 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 9ms/step - loss: 0.0222 - accuracy: 0.9588 - val_loss: 0.2136 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 9ms/step - loss: 0.0297 - accuracy: 0.9588 - val_loss: 0.1087 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 4/15
43/49 [=========================>....] - ETA: 0s - loss: 0.0642 - accuracy: 0.9070
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 10ms/step - loss: 0.0584 - accuracy: 0.9175 - val_loss: 0.0730 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 9ms/step - loss: 0.0088 - accuracy: 0.9691 - val_loss: 0.0306 - val_accuracy: 0.9167 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 9ms/step - loss: 0.0035 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## S6G Stan Matrix Model

In [ ]:
# Define the model
model_s6g_stan = keras.Sequential([
    keras.layers.Input(shape=(None, 400)),
    keras.layers.Dense(400, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(100, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(25, activation='relu',kernel_initializer='he_normal'),
    keras.layers.Dense(5, activation='sigmoid')
])

# Load your data and preprocess it
train_features = np.array(df['s6g_stan_matrix'].tolist())
train_labels = np.array(df[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].values)

train_features = train_features.reshape(-1, 400)

# Initialize K-Fold cross-validator
kf = KFold(n_splits=5)

# Initialize a list to store accuracy scores
accuracy_scores = []
precision_scores = []
recall_scores = []
acc_scores = []
acc_norm = []

# Iterate over each sample for LOOCV
for train_index, test_index in kf.split(train_features):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]

    # Compile the model and set up callbacks (as in your code)
    model_s6g_stan.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.00005, patience=5, verbose=1, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
    callbacks = [early_stopping, lr_scheduler]

    batch_size = 2
    model_s6g_stan.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=callbacks)

    # Make predictions on the current test sample
    y_pred = model_s6g_stan.predict(X_test)


    # Calculate and store the accuracy for this fold
    accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    precision = precision_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    recall = recall_score(y_test, (y_pred > 0.5).astype(int), average='weighted')
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    acc_scores.append(norm_accuracy(y_test, y_pred))
    acc_norm.append(calculate_accuracy(y_test, y_pred))

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_acc_norm = sum(acc_norm)/len(acc_norm)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_acc = sum(acc_scores)/len(acc_scores)
f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
grand_mean=(average_accuracy + avg_precision + avg_recall + f1 + avg_acc_norm + avg_acc)/6
# Print the average accuracy
print("Average Accuracy: ", average_accuracy)
print("Accuracy: ", avg_acc)
print("Average Normalized Accuracy: ", avg_acc_norm)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
print('F1 score:', f1)
print('Grand Mean:', grand_mean)

Epoch 1/15
48/48 [==============================] - 1s 9ms/step - loss: 0.6034 - accuracy: 0.4792 - val_loss: 0.4870 - val_accuracy: 0.4800 - lr: 0.0010
Epoch 2/15
48/48 [==============================] - 0s 5ms/step - loss: 0.3807 - accuracy: 0.6562 - val_loss: 0.3645 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 3/15
48/48 [==============================] - 0s 5ms/step - loss: 0.2939 - accuracy: 0.6875 - val_loss: 0.3916 - val_accuracy: 0.6400 - lr: 0.0010
Epoch 4/15
48/48 [==============================] - 0s 6ms/step - loss: 0.2312 - accuracy: 0.8125 - val_loss: 0.3351 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 5/15
48/48 [==============================] - 0s 5ms/step - loss: 0.1829 - accuracy: 0.8333 - val_loss: 0.3422 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 6/15
48/48 [==============================] - 0s 5ms/step - loss: 0.1602 - accuracy: 0.8438 - val_loss: 0.3158 - val_accuracy: 0.7600 - lr: 0.0010
Epoch 7/15
41/48 [========================>.....] - ETA: 0s - loss: 0.1172 - accur

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 8ms/step - loss: 0.2900 - accuracy: 0.7320 - val_loss: 0.2702 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.2000 - accuracy: 0.8454 - val_loss: 0.1295 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.1457 - accuracy: 0.8660 - val_loss: 0.1270 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 6ms/step - loss: 0.1451 - accuracy: 0.8041 - val_loss: 0.1271 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0972 - accuracy: 0.8660 - val_loss: 0.0940 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 6/15
46/49 [===========================>..] - ETA: 0s - loss: 0.0795 - accuracy: 0.9022
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 6ms/step - loss: 0.0789 - accura

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 8ms/step - loss: 0.1666 - accuracy: 0.8351 - val_loss: 0.1146 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.1032 - accuracy: 0.8866 - val_loss: 0.0764 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0685 - accuracy: 0.9072 - val_loss: 0.0881 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0835 - accuracy: 0.9278 - val_loss: 0.1204 - val_accuracy: 0.7917 - lr: 0.0010
Epoch 5/15
48/49 [============================>.] - ETA: 0s - loss: 0.0592 - accuracy: 0.9375
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 5ms/step - loss: 0.0587 - accuracy: 0.9381 - val_loss: 0.0906 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0228 - accura

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 2s 13ms/step - loss: 0.0613 - accuracy: 0.9691 - val_loss: 0.0306 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0781 - accuracy: 0.9278 - val_loss: 0.0890 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0422 - accuracy: 0.9381 - val_loss: 0.0183 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 4/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0250 - accuracy: 0.9485 - val_loss: 0.0395 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0185 - accuracy: 0.9691 - val_loss: 0.0135 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 6/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0196 - accuracy: 0.9588 - val_loss: 0.0247 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 7/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0258 - accuracy: 0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


49/49 [==============================] - 1s 9ms/step - loss: 0.0323 - accuracy: 0.9175 - val_loss: 0.0246 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0272 - accuracy: 0.9485 - val_loss: 0.0847 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 3/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0336 - accuracy: 0.9278 - val_loss: 0.0510 - val_accuracy: 0.9583 - lr: 0.0010
Epoch 4/15
39/49 [======================>.......] - ETA: 0s - loss: 0.0225 - accuracy: 0.9872
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
49/49 [==============================] - 0s 5ms/step - loss: 0.0307 - accuracy: 0.9794 - val_loss: 0.0338 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 5/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0046 - accuracy: 0.9381 - val_loss: 0.0089 - val_accuracy: 0.9583 - lr: 5.0000e-04
Epoch 6/15
49/49 [==============================] - 0s 5ms/step - loss: 0.0016 - ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
